<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

Changed

In [67]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
inPath = "../outputs/amazon_region/"
outPath = "../data/amazon_inference_data"

satalites = ["TERRA_M__M", "TERRA_M__T", "AQUA_M__M", "AQUA_M__T"]

nmonths_import = 16*12

## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [68]:
driver_paths_all = [os.path.join(dp, f) for (dp, _, fn) in os.walk(inPath) for f in fn if f.endswith('.nc')]
driver_names_all = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths_all]

file_table = pd.DataFrame({'filepath': driver_paths_all, 'file_name': driver_names_all})

def checkFilename(driver_name, sat):
    
    if "firecount" in driver_name:
        if sat in driver_name: return True
        return False
    
    return True

def select_files_for_sat(sat):
    test = [checkFilename(driver_name, sat) for driver_name in driver_names_all]
    
    driver_paths = np.array(driver_paths_all)[test]
    driver_names = np.array(driver_names_all)[test]
    
    driver_names = ["firecount" if "firecount" in driver_name else driver_name for driver_name in driver_names]
    return driver_paths, driver_names
    
driver_info = [select_files_for_sat(sat) for sat in satalites]

Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [69]:
from pdb import set_trace as browser

def nc_extract(fpath):
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['variable'][:,:,:])
        if (len(gdata) < nmonths_import): return None
        gdata = gdata[0:nmonths_import]
        gdata[gdata < -9E9] = np.nan
        gflat = gdata.flatten()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [70]:
values = [ [nc_extract(dp) for dp in driver_paths[0]] for driver_paths in driver_info]

Processing: ../outputs/amazon_region/GFED4s_2.5degree_2000-2016.nc
Processing: ../outputs/amazon_region/burnt_area-GFED4s_2.5degree_2000-2016.nc
Processing: ../outputs/amazon_region/vegetation/MaxOverMean_soilw.0-10cm.gauss.2000-2019.nc
Processing: ../outputs/amazon_region/vegetation/nonetreecover-2001-June2018.nc
Processing: ../outputs/amazon_region/vegetation/treecover-2001-June2018.nc
Processing: ../outputs/amazon_region/vegetation/vegcover-2001-June2018.nc
Processing: ../outputs/amazon_region/human/cropland2001-2019.nc
Processing: ../outputs/amazon_region/human/fract_agr2001-2019.nc
Processing: ../outputs/amazon_region/human/pasture2001-2019.nc
Processing: ../outputs/amazon_region/human/population_density-PD_HYDEv3.2_2001-2018.nc
Processing: ../outputs/amazon_region/climate/air2001-2019.nc
Processing: ../outputs/amazon_region/climate/emc-2001-2019.nc
Processing: ../outputs/amazon_region/climate/lightning2001-2019.nc
Processing: ../outputs/amazon_region/climate/relative_humidity2001

Turn this into a dataframe for the analysis.

In [71]:
# turn list into a dataframe
def makeDataFrave(value, driver_names):
    df = pd.DataFrame(np.array(value).T, columns=driver_names)
    print(df.info())
    df.dropna(inplace=True)
    return df
fire_df = [makeDataFrave(value, driver_names[1]) for value, driver_names in zip(values, driver_info)]
#fire_df.info()
# replace null flags with pandas null
#fire_df.replace(fire_df < -3e38, np.nan, inplace=True)
#fire_df[] = np.nan
# drop all null rows (are ocean and not needed in optim)
#fire_df.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71808 entries, 0 to 71807
Data columns (total 16 columns):
GFED                  38208 non-null float32
burnt_area            38208 non-null float32
MaxOverMean_soilw     38208 non-null float32
nonetreecover         38208 non-null float32
treecover             38208 non-null float32
vegcover              38208 non-null float32
cropland              38208 non-null float32
fract_agr             38208 non-null float32
pasture               38208 non-null float32
population_density    38208 non-null float32
air                   38208 non-null float32
emc                   38208 non-null float32
lightning             38208 non-null float32
relative_humidity     38208 non-null float32
soilw                 38208 non-null float32
firecount             38208 non-null float32
dtypes: float32(16)
memory usage: 4.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71808 entries, 0 to 71807
Data columns (total 16 columns):
GFED            

Check that we've built it correctly.

In [72]:
for df in fire_df:
    print(df.head(10))

        GFED  burnt_area  MaxOverMean_soilw  nonetreecover  treecover  \
25  0.000756    0.000756           1.170661       0.553620   0.312995   
26  0.001449    0.001449           1.249730       0.518482   0.313169   
27  0.004058    0.004058           1.293419       0.559596   0.237006   
28  0.006394    0.006394           1.286996       0.557182   0.246418   
29  0.003410    0.003410           1.274551       0.561806   0.181881   
46  0.000013    0.000013           1.036566       0.503342   0.389530   
47  0.000451    0.000451           1.038840       0.481445   0.441354   
48  0.002569    0.002569           1.126523       0.540335   0.360199   
49  0.011624    0.011624           1.163553       0.598704   0.266478   
50  0.019312    0.019312           1.219175       0.594417   0.240804   

    vegcover  cropland  fract_agr   pasture  population_density        air  \
25  0.866615  0.083788   0.590337  0.506548           54.569523  25.933542   
26  0.831651  0.090265   0.450214  0.359

Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [73]:
for sat, df in zip(satalites, fire_df):
    outPathi = outPath + '-' + sat + '.csv'
    savepath = os.path.expanduser(outPathi)
    df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>